https://dspguru.com/files/QuadSignals.pdf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from numpy.fft import fft, ifft, fftfreq, fftshift

from physdim import units as u
from physdim import constants as K
from physdim import PhysicalValue
PhysicalValue.repr_is_str = True

In [ ]:
T = 100*u.usec
fo = 850*u.kHz
BW = 400*u.kHz

In [ ]:
IPRr = 0.5*K.c/BW
IPRr

# Transmit Signal

$$
\begin{align}
f(t) &= f_s + \gamma\,T \qquad
\text{where}\quad f_s = f_o - \frac{BW}{2}\quad
\text{and}\quad\gamma = \frac{BW}{T} \\
\omega(t) &= 2\pi f(t) = 2\pi\left(f_s + \gamma\,T\right)\\
\\
\phi(t) &= \phi_o + \int_0^t \omega(\tau)\,d\tau = \phi_o + 2\pi\left(f_s\,t + \frac{\gamma\,t^2}{2}\right)\\
\\
\sigma_{tx}(t) &= \tau\left[0\le t\le T\right]\;\sin\left[\phi_o + 2\pi\left(f_s\,t + \frac{\gamma\,t^2}{2}\right)\right]
\end{align}
$$

In [ ]:
fs = fo-BW/2
gamma = BW/T
phi_o = 0

def xmit(t):
    if t<0*u.sec or t>T:
        return 0
    return np.sin(phi_o + 2*np.pi*(t*(fs+0.5*gamma*t)))    

In [ ]:
Fs = 20*fo
t = [i/Fs for i in range(int(Fs*T))]
x = [tt.usec for tt in t]
s = [xmit(tt) for tt in t]
plt.figure(figsize=(18,4))
plt.plot(x,s)
plt.show()

## Return Signal

In [ ]:
# List of signal reflectors
#  Each reflector is a tuple containing range and relative cross section
reflectors = (
    (500*u.nmi, 1.0),
    (503*u.nmi, 0.2),
    (510*u.nmi, 0.5),
    (490*u.nmi, 0.75),
)

def rcv(t):
    return sum( A * xmit(t-2*rs/K.c) for rs,A in reflectors)

In [ ]:
round_trip_times = tuple(2*Rs/K.c for Rs,_ in reflectors)
Tr_s = min(round_trip_times)
Tr_e = max(round_trip_times) + T
Tp_s = 1.05*Tr_s - 0.05*Tr_e
Tp_e = 1.05*Tr_e - 0.05*Tr_s
t = [Tp_s + i/Fs for i in range(int(Fs*(Tp_e-Tp_s)))]
x = [tt.usec for tt in t]
s = [rcv(tt) for tt in t]
plt.figure(figsize=(18,4))
plt.plot(x,s)
plt.show()

# Chirp

$$
\begin{align}
f_{chirp}(t) &= f_c \\
\phi_{chirp}(t) &= \phi_{oc} + 2\pi f_c\,t\\
\sigma_{chirp}(t) &= \sin\left(\phi_{oc} + 2\pi f_c\,t\right)
\end{align}
$$

In [ ]:
fc=fo
phi_oc = 0
def deramp_chirp(t):
    return np.sin(phi_oc + 2*np.pi*fc*t)

def rcv_chirp(t):
    return rcv(t)*deramp_chirp(t)

In [ ]:
s = [rcv_chirp(tt) for tt in t]
plt.figure(figsize=(18,4))
plt.plot(x,s)
plt.show()

In [ ]:
S = fft(s)
X = fftfreq(len(S),1/Fs)

In [ ]:
plt.figure(figsize=(18,4))
plt.plot(X.kHz,np.abs(S))
plt.show()

In [ ]:
f = np.abs(X)<1000*u.kHz
Sr = S[f]
Xr = X[f]

In [ ]:
plt.figure(figsize=(18,4))
plt.plot(Xr.kHz,np.abs(Sr))
plt.show()

In [ ]:
sr = np.real(ifft(Sr))
xr = fftshift(fftfreq(len(Sr),X[1]))*u.sec

In [ ]:
plt.figure(figsize=(18,4))
plt.plot(xr.usec,sr)
plt.show()

# Stretch

$$
\begin{align}
f_{stretch}(t) &= f_{so} + \gamma\,t \\
\phi_{stretch}(t) &= \phi_{os} + 2\pi f_{so}\,t + \pi\gamma\,t^2\\
\sigma_{chirp}(t) &= \sin\left(\phi_{os} + 2\pi f_{so}\,t + \pi\gamma\,t^2\right)
\end{align}
$$

In [ ]:
fso=fo-gamma*(0.5*(Tr_s+Tr_e)-Tp_s)
phi_os = 0

def deramp_stretch(t):
    return np.sin(phi_os + 2*np.pi*t*(fso + 0.5*gamma*t))

def rcv_stretch(t):
    return rcv(t)*deramp_stretch(t-Tp_s)

In [ ]:
s = [rcv_stretch(tt) for tt in t]
plt.figure(figsize=(18,4))
plt.plot(x,s)
plt.show()

In [ ]:
S = fft(s)
X = fftfreq(len(S),1/Fs)

In [ ]:
plt.figure(figsize=(18,4))
plt.plot(X.kHz,np.abs(S))
plt.show()

In [ ]:
f = np.abs(X)<1000*u.kHz
Sr = S[f]
Xr = X[f]

In [ ]:
plt.figure(figsize=(18,4))
plt.plot(Xr.kHz,np.abs(Sr))
plt.twinx()
plt.plot(Xr.kHz,np.angle(Sr),'.',color='orange')
plt.show()

In [ ]:
sr = np.real(ifft(Sr))
xr = fftshift(fftfreq(len(Sr),X[1]))*u.sec

In [ ]:
plt.figure(figsize=(18,4))
plt.plot(xr.usec,sr)
plt.show()

In [ ]:
g = np.exp(fftshift(np.arange(len(Sr))*(-2j*np.pi*100/len(Sr))))
Srd = Sr * g

In [ ]:
Srd = Sr * np.exp(1j*75*u.deg/u.rad)

In [ ]:
plt.figure(figsize=(18,4))
plt.plot(Xr.kHz,np.abs(Srd))
ax2 = plt.twinx()
ax2.plot(Xr.kHz,np.angle(Srd),'.',color='orange')
plt.show()

In [ ]:
srd = np.real(ifft(Srd))

In [ ]:
plt.figure(figsize=(18,4))
plt.plot(xr.usec,srd)
plt.show()